In [15]:
# All imports for this question
# from google.colab import userdata
import google.generativeai as genai
from datasets import Dataset
import random
from typing import Callable, List, Any
import json
import requests
import datetime
import pandas as pd

In [14]:
import json

GEMINI_api_key = ""
ChatGPT_key = ""
Finhub_key = ""
with open(r'../Keys/config.json') as config_file:
    config = json.load(config_file)
    GEMINI_api_key = config['Gemini_api_key']
    ChatGPT_key = config['GPT_Key']
    Finhub_key = config['Finhub_Key']


Initializing Gemini API

In [5]:
genai.configure(api_key=GEMINI_api_key)
model = genai.GenerativeModel("gemini-1.5-flash")
# model = genai.GenerativeModel("gemini-1.0-pro")
response = model.generate_content("You are a financial analyst. Based on the data I am sharing with you, I need you to predict whether the value is going to increase or decrease")
print(response.text)
# response = model.generate_content("Explain how AI works")
# print(response.text)

Imagine you have a special friend, a computer, who can understand your words just like your mommy and daddy! That's what **Natural Language Processing** is all about. It's like teaching the computer how to speak and understand our language, English.

It's like when you learn to read a book, you learn the words and how they fit together to make a story. The computer learns the words and how they fit together to understand what you're saying. Then, it can even answer you back in English!

So, when you talk to a computer using **Natural Language Processing**, it's like having a fun conversation with your friend! You can ask it questions, tell it stories, and even play games. It's amazing! 



Initializing ChatGPT API

In [12]:
from openai import OpenAI
client = OpenAI(api_key=ChatGPT_key)

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)

print(completion.choices[0].message.content)

Functions call themselves,  
Echoing in endless loops,  
Depths of code unfold.  


In [10]:
print(completion.choices)

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Code calls itself back,  \nLayers of thought intertwine,  \nEndless paths unfold.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))]


Working on the context

In [9]:
def GeminiCall(company, History = None, News = None ,Financial = None,Twitter = None):
    context = "You are a financial analyst. Analyze the following data and predict whether the stock price will increase or decrease:\n\n"
    data = f"**Company:**{company} \n**Historical Stock Price:**{History}\n**Recent News:**{None}\n**Financial Statements:**{Financial}\n**Social Media Sentiment:**{Twitter}"
    prompt  = context + data + "\n\n\n Make the Prediction: (increase or decrease)"
    response = model.generate_text(prompt=prompt)
    print(response)

In [21]:
def GPTCall(prompt):
    # prompt = f"**Company:**{company} \n**Historical Stock Price:**{History}\n**Recent News:**{None}\n**Financial Statements:**{Financial}\n**Social Media Sentiment:**{Twitter}"
    # Maybe create a Prompt function
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a financial analyst. Analyze the following data and respond with only one word: either 'increase' or 'decrease' to indicate the stock price trend.:\n\n"},
            {"role": "user", "content": prompt}
        ]
    )
    prediction = response.choices[0].message.content.strip()
    return prediction

In [22]:
def create_prompt(ticker, date_str, news_data, stock_data):
    # prompt = f"Please analyze the following data for {ticker} up to {date_str} and predict whether the stock price will go up or down:\n\n"
    # prompt = f"**Company:**{ticker} \n**Date of prediction:**{date_str} \n**Historical Stock Price:**{History}\n**Recent News:**{None}\n**Financial Statements:**{Financial}\n**Social Media Sentiment:**{Twitter}"
    prompt = f"Company: {ticker}\n Date of prediction: {date_str}\n"
    prompt += "Recent News Headlines:\n"
    for news in news_data[:5]:  # Limit to first 5 news articles
        prompt += f"- {news['headline']}\n"

    prompt += "\nStock Market Data (last 30 days):\n" # This is not correct right?
    # Only select relevant columns
    stock_info = stock_data[
        ["Date", "Stock Price", "High", "Low", "Close", "News Count"]
    ]
    prompt += stock_info.to_string(index=False)

    # We should include the twitter sentiment involved here

    return prompt

In [23]:
def get_company_news(ticker, from_date, to_date):
    """
    Fetch news articles for a company from the Finnhub API.
    """
    url = 'https://finnhub.io/api/v1/company-news'
    params = {
        'symbol': ticker,
        'from': from_date,
        'to': to_date,
        'token': Finhub_key
    }
    response = requests.get(url, params=params)
    data = response.json()
    return data

def prepare_data(ticker, date_str):
    """
    Prepare news and stock data for the given company and date.
    """
    # Parse date in DD/MM/YYYY format and reformat it to YYYY-MM-DD
    date = datetime.datetime.strptime(date_str, '%d/%m/%Y')
    from_date = (date - datetime.timedelta(days=30)).strftime('%Y-%m-%d')
    to_date = date.strftime('%Y-%m-%d')
    
    news_data = get_company_news(ticker, from_date, to_date)
    
    stock_data = pd.read_csv(r'../Dataset/stock_data_august_2024.csv')
    # Filter stock data for the company and last 30 days
    stock_data['Date'] = pd.to_datetime(stock_data['Date'], dayfirst=True)

    mask = (stock_data['Ticker'] == ticker) & (stock_data['Date'] >= from_date) & (stock_data['Date'] <= to_date)
    company_stock_data = stock_data.loc[mask]
    
    return news_data, company_stock_data


def main():
    ticker = input("Input the company name:")
    date_str = input("Enter date in DD/MM/YYYY format:")  # Example date in DD/MM/YYYY format
    
    # Prepare data
    news_data, company_stock_data = prepare_data("MSFT", "10/10/2024")
    # print(news_data)
    
    # Create prompt
    prompt = create_prompt(ticker, date_str, news_data, company_stock_data)
    
    # Get prediction
    prediction = GPTCall(prompt)
    
    # Display the prediction
    print("Prediction:\n")
    print(prediction)

if __name__ == "__main__":
    main()

Prediction:

decrease
